In [4]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# results save path
base_result_path = '../results/METR-LA/LSTM'
exp_name = 'multivariate_AE_weighted_vector_real_time_results.pkl'
results_save_path = os.path.join(base_result_path, exp_name)

In [ ]:
columns_names = ['717508','772151','767366','769405','769388','716951','717497','717483','717468','772596','772597','717469','764120','772178','769941','760024','771673','771667','765604','765176','773927','717480','717456','717495','716953','717481','767572','718371','769403','769359','716943','717491','767610','717446','717452','717453','717447','716956','717490','716942','773880','718204','769358','769402','767573','769953','773062','765164','772140','772168','769372','716571','716968','717492','717486','769819','717445','773869','769831','717450','716941','716955','717493','764858','769373','765171','768469','773012','717580','773953','763995','718499','716339','718089','718076','717608','769467','772669','717595','773013','717583','759772','764424','717804','764949','717582','773996','717592','759602','764781','767471','718064','767470','765273','764794','773954','717578','717587','773995','717585','769926','716328','765265','761003','718072','769847','718066','761599','765099','760650','717816','761604','717590','774012','717576','774204','767495','767494','759591','716331','762329','767523','767053','773024','774011','764766','767509','717818','767455','769444','716939','717825','767454','717819','773974','717570','718496','717821','737529','718045','716337','717571','773975','717573','764760','769867','718090','772513','717610','717823','769443','717572','773023','773939','767541','769418','769430','716958','717461','717460','716554','769431','764101','767554','773904','765182','718141','760987','717502','773906','767542','772167','717099','767350','767620','717489','717462','717463','767609','717488','767621','767351','774067','767751','717513','764106','768066','764853','717498','717473','717472','769806','717466','717499','769345','767585','718379','773916','767750','717504','717510','716949','717458','717465','717459','716960','769346']

In [ ]:
with open(results_save_path, 'rb') as f:
    results1 = pickle.load(f)

In [ ]:
org_df = pd.DataFrame(results1[0]['df']['Real'],columns=columns_names)
pred_df = pd.DataFrame(results1[0]['df']['Predicted'],columns=columns_names)

In [ ]:
sqrt(mean_squared_error(org_df['773062'].values,pred_df['773062'].values))

In [ ]:
org_df['717445']

# Real-Time result Analysis

In [75]:
target = '737529'
threshold = 0.05
time_window = 24

In [76]:
base_result_path = '../results/METR-LA/real_time_modeling/'
exp_name = f'univariate_AE_real_time_processing_threshold_{threshold}_{time_window}H_results_{target}.pkl'
results_load_path = os.path.join(base_result_path, exp_name)

In [77]:
# load results
with open(results_load_path, 'rb') as f:
    results_real_time = pickle.load(f)

In [78]:
results_real_time[target][threshold].keys()

dict_keys(['incremental_weighted_update', 'incremental_update', 'No_update', 'base_model'])

In [79]:
# get the average rmse and mae from all the experiments
def get_results(results_real_time, target, threshold, exp = 'No_update'):
    rmse = []
    mae = []
    train_time = []
    for key,val in results_real_time[target][threshold][exp].items():
        rmse.append(val['RMSE'])
        mae.append(val['MAE'])
        if exp != 'No_update':
            train_time.append(val['train_time'])
        else:
            train_time.append(0)
    print(f'RMSE: {np.mean(rmse)}')
    print(f'MAE: {np.mean(mae)}')
    print(f'Train time: {np.mean(train_time)}')
    # return rmse, mae, train_time

In [80]:
get_results(results_real_time, target, threshold, exp = 'incremental_weighted_update')

RMSE: 12.044180856281702
MAE: 7.845166206359863
Train time: 0.021019895197981496


In [81]:
get_results(results_real_time, target, threshold, exp = 'incremental_update')

RMSE: 12.00008107819342
MAE: 7.815212726593018
Train time: 0.01784764710119215


In [82]:
get_results(results_real_time, target, threshold, exp = 'No_update')

RMSE: 11.296915006448161
MAE: 7.359277725219727
Train time: 0.0


In [72]:
get_results(results_real_time, target, threshold, exp = 'incremental_weighted_update')

RMSE: 12.335480324885852
MAE: 10.86701488494873
Train time: 0.005507672962389494


In [73]:
get_results(results_real_time, target, threshold, exp = 'incremental_update')

RMSE: 12.402022270190002
MAE: 10.887054443359375
Train time: 0.005451359497873407


In [74]:
get_results(results_real_time, target, threshold, exp = 'No_update')

RMSE: 16.142935782701766
MAE: 12.458470344543457
Train time: 0.0


In [56]:
get_results(results_real_time, target, threshold, exp = 'incremental_weighted_update')

RMSE: 11.751373090139266
MAE: 10.418865203857422
Train time: 0.005428610851890162


In [57]:
get_results(results_real_time, target, threshold, exp = 'incremental_update')

RMSE: 11.72788030884934
MAE: 10.371858596801758
Train time: 0.00535873764439633


In [58]:
get_results(results_real_time, target, threshold, exp = 'No_update')

RMSE: 15.328964989254983
MAE: 12.401187896728516
Train time: 0.0
